# Problem 6

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.ticker as ticker   


# setup spark
conf = SparkConf().setAppName('MarkovShakespeare')
sc = SparkContext(conf=conf)

In [2]:
# load textlist from net
filename = "pg100.txt"

url = "http://www.gutenberg.org/cache/epub/100/pg100.txt"

import urllib

wordfile = urllib.URLopener()
wordfile.retrieve(url, filename)

('pg100.txt', <httplib.HTTPMessage instance at 0x1133bf6c8>)

In [3]:
# the texts from shakespeare are contained in lines 182 - 124367
# create the file Shakespeare.txt using the line range from above
# if all is ok, this should return True, True
with open(filename, 'r') as f:
    lines = f.readlines()
    lines = lines[181:124367]
    with open('Shakespeare.txt', 'w') as f2:
        f2.writelines(lines)
    print f2.closed
f.closed

True


True

In [347]:
# filtering is done by spark
words = sc.textFile('Shakespeare.txt')

In [348]:
# split lines into words (after whitespace)
# before splitting replace first some escape characters with white space
rdd = words.flatMap(lambda line: line.replace('\n', ' ').replace('\r', ' ').split(' '))

In [349]:
# now perform filtering on the words
# to develop the regular expression this awesome tool was used https://regex101.com

# (1) filter out words that only contain numbers
# (2) filter out words for which all letters are capitalized
# (3) filter out words that contain letters only and end with a period
import re

pattern1 = re.compile(ur'([0-9]+\.?)|\.')  # use only a simple RE for numbers here (numbers in the shakespeare text are formateed as 1., 2., ...)
pattern2 = re.compile(ur'(\b[A-Z]+\.?\b)|\.') # ==> do (2) & (3) together in one regex!

# filter does not change the order

# FILTERING DOES NOT WORK YET GOOD!

# first pattern works, second has troubles!
rdd = rdd.filter(lambda x: pattern1.match(x) is None).filter(lambda x: pattern2.match(x) is None).filter(lambda x: len(x) > 0)

In [350]:
rdd.take(5)

[u'From', u'fairest', u'creatures', u'we', u'desire']

In [351]:
# zip with index
rdd = rdd.zipWithIndex()

In [352]:
rdd.take(5)

[(u'From', 0), (u'fairest', 1), (u'creatures', 2), (u'we', 3), (u'desire', 4)]

In [353]:
rdd.map(lambda x:x).lookup(u'Now')

[231,
 1679,
 2628,
 8288,
 8303,
 9921,
 11886,
 12226,
 15578,
 18590,
 20856,
 20869,
 30398,
 30641,
 30703,
 30970,
 32002,
 37716,
 39790,
 41863,
 43033,
 45639,
 53116,
 53378,
 55072,
 55585,
 55796,
 56124,
 58337,
 59529,
 62304,
 62646,
 62695,
 62924,
 64717,
 65302,
 67966,
 72392,
 76689,
 76709,
 76734,
 78717,
 87425,
 89825,
 97288,
 101984,
 102526,
 105785,
 105978,
 106896,
 107666,
 109396,
 112407,
 113437,
 115209,
 115612,
 116751,
 117167,
 118609,
 119434,
 132951,
 133281,
 134639,
 136073,
 139888,
 144384,
 150076,
 153394,
 153465,
 157526,
 158043,
 160556,
 163346,
 163594,
 164038,
 165763,
 166266,
 167516,
 169164,
 169828,
 175563,
 177137,
 178671,
 179143,
 182482,
 184731,
 189301,
 189467,
 189588,
 196925,
 197002,
 205158,
 209234,
 209261,
 216733,
 218172,
 218717,
 227746,
 230690,
 230715,
 230918,
 235930,
 236026,
 236665,
 236681,
 237222,
 237281,
 238051,
 239600,
 240967,
 241433,
 244106,
 245124,
 246651,
 246685,
 254591,
 254817,

In [354]:
# map to list
rdd = rdd.flatMap(lambda x: [(x[1], (0, x[0])), (x[1] - 1, (1, x[0])), (x[1] - 2, (2, x[0]))])

In [355]:
#rdd.take(5)

In [356]:
rdd = rdd.groupByKey()

In [357]:
#rdd.take(5)

In [358]:
# map (keyA, (keyA, wordA), (keyA + 1, wordA+1), (keyA + 2, wordA+2)) to (wordA, wordA+1, wordA+2)
fun = lambda x: list(x[1])
rdd = rdd.map(fun)

In [359]:
# HIER NOCH UNBEDINGT ALLE TUPEL RAUSSCHMEISSEN, DEREN LISTE != 3 Elemente enthaelt
rdd = rdd.filter(lambda x: len(x) == 3)

In [360]:
# map such that the three words form the key with value 1, we use that then to sum up everything
#gfun = lambda x: ((x[0][1], x[1][1], x[2][1]), 1)

# try to combine
gfun = lambda x: ((x[0][1], x[1][1], x[2][1]), 1)
#gfun = lambda x: (x[0][1], 1)
#rdd = rdd.map(gfun)

In [361]:
rdd = rdd.map(gfun)

In [362]:
#temp.take(20)

In [363]:
rdd = rdd.reduceByKey(lambda x,y:x+y)
#temp.take(5)

In [364]:
# remap form
rdd = rdd.map(lambda x: ((x[0][0], x[0][1]), (x[0][2], x[1])))

In [365]:
# final group by key
# maybe sort list after it!!!
rdd = rdd.groupByKey().map(lambda x: (x[0],list(x[1]) ))

In [366]:
rdd.map(lambda x: x).lookup((u'Now', u'is'))

[[(u'a', 1),
  (u'my', 2),
  (u'the', 9),
  (u'this', 1),
  (u'that', 1),
  (u'be', 1),
  (u'it', 3),
  (u'he', 1),
  (u'his', 1),
  (u'your', 1),
  (u'Mortimer', 1)]]